!pip install pymongo

In [5]:
import os
import sys
sys.path.append(os.path.abspath('.\Common'))

In [6]:
import pandas as pd
import numpy as np
import datetime

import naver
import mongodb

In [7]:
mongo = mongodb.da()

In [8]:
cl = mongo.client.asset.list_collection_names()
cl = pd.Series(cl)
cl[(cl.str.contains('연간')==False) & (cl.str.contains('분기')==False) & (cl.str.contains('펀더멘털')==False)]

140         code
440     corp_fin
1436       kospi
2475    interest
2759      dollar
2886      kosdaq
3315         dow
5044         oil
dtype: object

In [9]:
def init_save_db():
    naver_stock = naver.stock_da()
    naver_index = naver.index_da()    
    
    kospi = naver_index.get_ts_data_kor_index('kospi', 1315)
    mongo.client.asset.kospi.drop()
    mongo.client.asset.kospi.insert_many(kospi.reset_index().to_dict('records'))
    
    kosdaq = naver_index.get_ts_data_kor_index('kosdaq', 995)
    mongo.client.asset.kosdaq.drop()
    mongo.client.asset.kosdaq.insert_many(kosdaq.reset_index().to_dict('records'))
    
    dow = naver_index.get_ts_data_world_index('dow', 460)
    mongo.client.asset.dow.drop()
    mongo.client.asset.dow.insert_many(dow.reset_index().to_dict('records'))
    
    dollar = naver_index.get_ts_data_exchange('$', 398)
    mongodb.client.asset.dollar.drop()
    mongo.client.asset.dollar.insert_many(dollar.reset_index().to_dict('records'))

    oil = naver_index.get_ts_data_oil('oil', 472)
    mongo.client.asset.oil.drop()
    mongo.client.asset.oil.insert_many(oil.reset_index().to_dict('records'))

    interest = naver_index.get_ts_data_interest('interest', 682)
    mongo.client.asset.interest.drop()
    mongo.client.asset.interest.insert_many(interest.reset_index().to_dict('records'))

    for i, row in naver_stock.code_df.iterrows():
        print(i,row['name'], row['code'])    

        mongo.client.asset[row['code']+'_연간'].drop()
        연간 = naver_stock.get_종합정보_기업실적분석_연간(code)
        mongo.client.asset[row['code']+'_연간'].insert_many(연간.reset_index().to_dict('records'))

        mongo.client.asset[row['code']+'_분기'].drop()
        분기 = naver_stock.get_종합정보_기업실적분석_분기(code)
        mongo.client.asset[row['code']+'_분기'].insert_many(분기.reset_index().to_dict('records'))    

In [10]:
def insert_new_data():    
    naver_index = naver.index_da() 
    
    mongo.insert_new(mongo.client.asset.kospi, naver_index.get_ts_data_kor_index('kospi', 2), '날짜')
    mongo.insert_new(mongo.client.asset.kosdaq, naver_index.get_ts_data_kor_index('kosdaq', 2), '날짜')
    
    mongo.insert_new(mongo.client.asset.dow, naver_index.get_ts_data_world_index('dow', 2), 'xymd')
    
    mongo.insert_new(mongo.client.asset.dollar, naver_index.get_ts_data_exchange('$', 2), '날짜')
    
    mongo.insert_new(mongo.client.asset.oil, naver_index.get_ts_data_oil('oil', 2), '날짜')
    
    mongo.insert_new(mongo.client.asset.interest, naver_index.get_ts_data_interest('interest', 2), '날짜')    

In [11]:
# insert_new_data()

요청 URL = https://finance.naver.com/sise/sise_index_day.nhn?code=kospi
요청 URL = https://finance.naver.com/sise/sise_index_day.nhn?code=kosdaq
요청 URL = https://finance.naver.com//world/worldDayListJson.nhn?symbol=DJI@DJI&fdtc=0
요청 URL = https://finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_USDKRW
요청 URL = https://finance.naver.com/marketindex/worldDailyQuote.nhn?marketindexCd=OIL_CL&fdtc=2
요청 URL = https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CD91


In [17]:
mongo.client.asset['001070'+'_펀더멘털'].find_one()

{'2018/12(A)': '8.01',
 '2019/12(E)': nan,
 '_id': ObjectId('5e7edf3071ece9b84fec01b7'),
 '주요지표': 'PER'}

In [18]:
mongo.collection_to_df(mongo.client.asset['001070'+'_펀더멘털'], '주요지표')

,2018/12(A),2019/12(E)
주요지표,,
회계기준,연결,연결
현금배당수익률,1.58%,NaN
현금DPS,300원,NaN
PER,8.01,NaN
PCR,-5.37,NaN
PBR,0.55,NaN
EV/EBITDA,-4.66,NaN
EPS,"2,365원",NaN
EBITDA,-112.9억원,NaN


In [10]:
naver_stock = naver.stock_da()

for i, row in naver_stock.code_df.iloc[265:].iterrows():
    print(i,row['name'], row['code'])    

    mongo.client.asset[row['code']+'_펀더멘털'].drop()
    펀터멘털 = naver_stock.get_펀더멘털(row['code'])
    if 펀터멘털 is not None:
        mongo.client.asset[row['code']+'_펀더멘털'].insert_many(펀터멘털.to_dict('records'))
    

265 해마로푸드서비스 220630
266 현대통신 039010
267 화이브라더스코리아 204630
268 휴맥스 115160
269 흥국 010240
270 LPK로보틱스 183350
271 나눔테크 244880
272 다이오진 271850
273 선바이오 067370
274 시그넷이브이 260870
275 에듀파트너 208890
276 에스알바이오텍 270210
277 에스엘에스바이오 246250
278 에프앤가이드 064850
279 엔에스엠 238170
280 옐로페이 179720
281 유투바이오 221800
282 이도바이오 336040
283 인카금융서비스 211050
284 전우정밀 120780
285 지놈앤컴퍼니 314130
286 진코스텍 250030
287 큐엠씨 136660
288 티케이씨 191600
289 티티씨디펜스 309900
290 파워풀엑스 266870
291 AJ네트웍스 095570
292 BGF리테일 282330
293 CJ CGV 079160
294 DSR제강 069730
295 GKL 114090
296 GS건설 006360
297 KB금융 105560
298 SKC 011790
299 금호산업 002990
300 금호전기 001210
301 기업은행 024110
302 남선알미늄 008350
303 남성 004270
304 대구백화점 006370
305 대덕전자 008060
306 대우건설 047040
307 대원제약 003220
308 대한방직 001070
309 대한제당 001790
310 대현 016090
311 동방아그로 007590
312 동부제철 016380
313 동성제약 002210
314 동원금속 018500
315 두산인프라코어 042670
316 디씨엠 024090
317 디티알오토모티브 007340
318 락앤락 115390
319 롯데손해보험 000400
320 롯데칠성음료 005300
321 만도 204320
322 미래에셋생명 085620
323 미원홀딩스 107590
324 백산 03515

753 룽투코리아 060240
754 리드 197210
755 리드코프 012700
756 머큐리 100590
757 메디포스트 078160
758 모바일리더 100030
759 바이오니아 064550
760 보라티알 250000
761 비아트론 141000
762 삼륭물산 014970
763 삼보판지 023600
764 삼본전자 111870
765 삼천리자전거 024950
766 상아프론테크 089980
767 서부T&D 006730
768 서진오토모티브 122690
769 서한 011370
770 세운메디칼 100700
771 세종텔레콤 036630
772 소프트캠프 258790
773 솔본 035610
774 수산아이앤티 050960
775 슈피겐코리아 192440
776 시공테크 020710
777 신성델타테크 065350
778 신한제5호스팩 323230
779 신한제6호스팩 333050
780 신화콘텍 187270
781 실리콘웍스 108320
782 쎄노텍 222420
783 쎄미시스코 136510
784 쎄트렉아이 099320
785 쎌바이오텍 049960
786 씨엠에스에듀 225330
787 씨큐브 101240
788 아리온 058220
789 아비코전자 036010
790 아이스크림에듀 289010
791 아이엘사이언스 307180
792 아이엠 101390
793 아이원스 114810
794 아프리카TV 067160
795 알파홀딩스 117670
796 애니젠 196300
797 에스마크 030270
798 에스앤더블류 103230
799 에스에프에이 056190
800 에스엠코어 007820
801 에이디칩스 054630
802 에이비엘바이오 298380
803 에이스침대 003800
804 에이스토리 241840
805 에이치시티 072990
806 에이텍티앤 224110
807 엑셈 205100
808 엔피디 198080
809 엘비세미콘 061970
810 엘엠에스 073110
811 엘오티베큠 083310
812 엘컴텍 03795

1231 수산중공업 017550
1232 신세계I&C 035510
1233 신세계건설 034300
1234 신영와코루 005800
1235 신원 009270
1236 아모레퍼시픽그룹 002790
1237 에스제이엠 123700
1238 에쓰씨엔지니어링 023960
1239 에이블씨엔씨 078520
1240 에이엔피 015260
1241 에이프로젠 KIC 007460
1242 오리온 271560
1243 우리금융지주 316140
1244 우성사료 006980
1245 우진아이엔에스 010400
1246 웰바이오텍 010600
1247 이리츠코크렙 088260
1248 이월드 084680
1249 인천도시가스 034590
1250 일동홀딩스 000230
1251 일진머티리얼즈 020150
1252 일진전기 103590
1253 조광피혁 004700
1254 조일알미늄 018470
1255 지엠비코리아 013870
1256 진흥기업 002780
1257 카카오 035720
1258 케이씨티시 009070
1259 코스맥스비티아이 044820
1260 태림포장 011280
1261 태원물산 001420
1262 티에이치엔 019180
1263 팬오션 028670
1264 평화홀딩스 010770
1265 폴루스바이오팜 007630
1266 하나투어 039130
1267 한국타이어앤테크놀로지 161390
1268 한국프랜지공업 010100
1269 한국화장품제조 003350
1270 한독 002390
1271 한세엠케이 069640
1272 한세예스24홀딩스 016450
1273 한솔제지 213500
1274 한솔케미칼 014680
1275 한솔테크닉스 004710
1276 한온시스템 018880
1277 한전산업 130660
1278 한진 002320
1279 해성디에스 195870
1280 현대상선 011200
1281 현대종합상사 011760
1282 현대해상 001450
1283 혜인 003010
1284 휴니드테크놀러지스 005870
1285 휴비스 079980

URLError: <urlopen error [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다>

In [95]:
# insert_new_data()

요청 URL = https://finance.naver.com/sise/sise_index_day.nhn?code=kospi
요청 URL = https://finance.naver.com/sise/sise_index_day.nhn?code=kosdaq
요청 URL = https://finance.naver.com//world/worldDayListJson.nhn?symbol=DJI@DJI&fdtc=0
요청 URL = https://finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_USDKRW
요청 URL = https://finance.naver.com/marketindex/worldDailyQuote.nhn?marketindexCd=OIL_CL&fdtc=2
요청 URL = https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CD91


In [97]:
mongo.collection_to_df(mongo.client.asset.kospi, '날짜').head()

,거래대금(백만),거래량(천주),등락률,전일비,체결가
날짜,,,,,
2020-03-27,12732416.0,1067933.0,+1.87%,31.49,1717.73
2020-03-26,12374197.0,992473.0,-1.09%,18.52,1686.24
2020-03-25,12612287.0,872808.0,+5.89%,94.79,1704.76
2020-03-24,10481447.0,679288.0,+8.60%,127.51,1609.97
2020-03-23,9645271.0,647528.0,-5.34%,83.69,1482.46


In [98]:
mongo.collection_to_df(mongo.client.asset.dow, 'xymd').head()

,clos,diff,gvol,high,low,open,rate,symb
xymd,,,,,,,,
2020-03-26,22552.17,1351.62,713734739,22595.06,21427.10,21468.38,6.38,DJI@DJI
2020-03-25,21200.55,495.64,803309664,22019.93,20538.34,21050.34,2.39,DJI@DJI
2020-03-24,20704.91,2112.98,808466182,20737.70,19649.25,19722.19,11.37,DJI@DJI
2020-03-23,18591.93,-582.05,801235525,19121.01,18213.65,19028.36,-3.04,DJI@DJI
2020-03-20,19173.98,-913.21,880374321,20531.26,19094.27,20253.15,-4.55,DJI@DJI


In [60]:
# dollar = naver.get_ts_data_exchange('$', 398)
# naver.mongo.client.asset.dollar.insert_many(dollar.reset_index().to_dict('records'))

# oil = naver.get_ts_data_oil('oil', 472)
# naver.mongo.client.asset.oil.insert_many(oil.reset_index().to_dict('records'))

# interest = naver.get_ts_data_interest('interest', 682)
# naver.mongo.client.asset.interest.insert_many(interest.reset_index().to_dict('records'))

# kosdaq = naver.get_ts_data_kor_index('kosdaq', 995)
# kospi = naver.get_ts_data_kor_index('kospi', 1315)

# naver.mongo.client.asset.kospi.insert_many(kospi.reset_index().to_dict('records'))
# naver.mongo.client.asset.kosdaq.insert_many(kosdaq.reset_index().to_dict('records'))

# for i, row in naver.code_df.iterrows():
#     print(i,row['name'], row['code'])    
    
#     naver.mongodb.client.asset[row['code']+'_연간'].drop()
#     연간 = naver.get_종합정보_기업실적분석_연간(code)
#     naver.mongodb.client.asset[row['code']+'_연간'].insert_many(연간.reset_index().to_dict('records'))
    
#     naver.mongodb.client.asset[row['code']+'_분기'].drop()
#     분기 = naver.get_종합정보_기업실적분석_분기(code)
#     naver.mongodb.client.asset[row['code']+'_분기'].insert_many(분기.reset_index().to_dict('records'))    

# mongo.insert_new(mongo.client.asset.dollar, naver.get_ts_data_exchange('$', 2), '날짜')
# mongo.insert_new(mongo.client.asset.kospi, naver.get_ts_data_kor_index('kospi', 2), '날짜')
# mongo.insert_new(mongo.client.asset.kospi, naver.get_ts_data_kor_index('kosdaq', 2), '날짜')

# mongo.client.asset.kospi.remove({'날짜': { "$gt" :datetime.datetime(2020, 3, 18, 0, 0)} })

In [73]:
mongo.collection_to_df(mongo.client.asset.kosdaq, '날짜').head(20)
# 
# pd.to_numeric(dollar.매매기준율).sort_index().iloc[-20:].plot()

,index,거래대금(백만),거래량(천주),등락률,전일비,체결가
날짜,,,,,,
2020-03-26,0.0,11902864.0,1649355.0,+2.16%,10.93,516.61
2020-03-25,1.0,9686607.0,1441082.0,+5.26%,25.28,505.68
2020-03-24,2.0,6742315.0,1143642.0,+8.26%,36.64,480.40
2020-03-23,3.0,6163699.0,1086690.0,-5.13%,23.99,443.76
2020-03-20,NaN,6503974.0,1243149.0,+9.20%,39.40,467.75
2020-03-20,4.0,6503974.0,1243149.0,+9.20%,39.40,467.75
2020-03-19,NaN,8575181.0,1646248.0,-11.71%,56.79,428.35
2020-03-19,5.0,8575181.0,1646248.0,-11.71%,56.79,428.35
2020-03-18,NaN,7410179.0,1322271.0,-5.75%,29.59,485.14


In [38]:
naver.get_ts_data_kor_index('kosdaq', 2)

요청 URL = https://finance.naver.com/sise/sise_index_day.nhn?code=kosdaq


,체결가,전일비,등락률,거래량(천주),거래대금(백만)
날짜,,,,,
2020-03-26,516.61,10.93,+2.16%,1649355.0,11902864.0
2020-03-25,505.68,25.28,+5.26%,1441082.0,9686607.0
2020-03-24,480.40,36.64,+8.26%,1143642.0,6742315.0
2020-03-23,443.76,23.99,-5.13%,1086690.0,6163699.0
2020-03-20,467.75,39.40,+9.20%,1243149.0,6503974.0
2020-03-19,428.35,56.79,-11.71%,1646248.0,8575181.0


In [35]:
mongo.collection_to_df(mongo.client.asset.kosdaq, '날짜').head(20)

,index,거래대금(백만),거래량(천주),등락률,전일비,체결가
날짜,,,,,,
2020-03-26,0.0,11902864.0,1649355.0,+2.16%,10.93,516.61
2020-03-25,1.0,9686607.0,1441082.0,+5.26%,25.28,505.68
2020-03-24,2.0,6742315.0,1143642.0,+8.26%,36.64,480.40
2020-03-23,3.0,6163699.0,1086690.0,-5.13%,23.99,443.76
2020-03-20,NaN,6503974.0,1243149.0,+9.20%,39.40,467.75
2020-03-20,4.0,6503974.0,1243149.0,+9.20%,39.40,467.75
2020-03-19,NaN,8575181.0,1646248.0,-11.71%,56.79,428.35
2020-03-19,5.0,8575181.0,1646248.0,-11.71%,56.79,428.35
2020-03-18,NaN,7410179.0,1322271.0,-5.75%,29.59,485.14
